# Data Analysis for alpha01kappa10

This notebook analyzes the simulation data from the alpha01kappa10 directory.

## Simulation Parameters
- Grid: 512×512
- Domain: 64×64
- Alpha: 0.1
- Kappa: 1.0
- Time step: 0.005
- Frames: 5000

## Data Files
- `den.h5` - Density data
- `phi.h5` - Potential data
- `w.h5` - Vorticity data
- `restart.h5` - Restart/state data


In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Try to import h5py, install if needed
try:
    import h5py
    print(f"h5py version: {h5py.__version__}")
except ImportError:
    print("h5py not found. Installing...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "h5py", "--user"])
    import h5py
    print("h5py installed successfully")

# Set up plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Set data directory
data_dir = "/global/cfs/cdirs/m4466/public/MHW/data/alpha01kappa10"
print(f"Data directory: {data_dir}")
print(f"Files in directory: {os.listdir(data_dir)}")


h5py version: 3.10.0
Data directory: /global/cfs/cdirs/m4466/public/MHW/data/alpha01kappa10
Files in directory: ['restart.h5', 'phi.h5', 'w.h5', 'phw.in', 'den.h5']


## Explore HDF5 File Structure


In [2]:
def explore_h5_file(filename):
    """Explore the structure of an HDF5 file."""
    filepath = os.path.join(data_dir, filename)
    if not os.path.exists(filepath):
        print(f"File {filename} does not exist or is empty")
        return None
    
    try:
        with h5py.File(filepath, 'r') as f:
            print(f"\n=== {filename} ===")
            print(f"Keys: {list(f.keys())}")
            
            def print_structure(name, obj):
                if isinstance(obj, h5py.Dataset):
                    print(f"  Dataset: {name}")
                    print(f"    Shape: {obj.shape}")
                    print(f"    Dtype: {obj.dtype}")
                    if obj.size > 0:
                        print(f"    Min: {np.min(obj[:]):.6e}, Max: {np.max(obj[:]):.6e}")
                elif isinstance(obj, h5py.Group):
                    print(f"  Group: {name}")
            
            f.visititems(print_structure)
            return f
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

# Explore all HDF5 files
files_to_explore = ['den.h5', 'phi.h5', 'w.h5', 'restart.h5']
for filename in files_to_explore:
    explore_h5_file(filename)



=== den.h5 ===
Keys: ['den']
  Dataset: den
    Shape: (5001, 512, 512)
    Dtype: float64
    Min: -3.884992e+01, Max: 3.602942e+01

=== phi.h5 ===
Keys: ['phi']
  Dataset: phi
    Shape: (5001, 512, 512)
    Dtype: float64
    Min: -3.549513e+01, Max: 3.670573e+01

=== w.h5 ===
Keys: ['w']
  Dataset: w
    Shape: (5001, 512, 512)
    Dtype: float64
    Min: -6.138263e+01, Max: 6.129746e+01

=== restart.h5 ===
Keys: ['den', 'deni', 'w', 'wi']
  Dataset: den
    Shape: (512, 512)
    Dtype: float64
    Min: -2.058500e+01, Max: 2.123511e+01
  Dataset: deni
    Shape: (512, 512)
    Dtype: float64
    Min: -2.058860e+01, Max: 2.123600e+01
  Dataset: w
    Shape: (512, 512)
    Dtype: float64
    Min: -3.135387e+01, Max: 3.826308e+01
  Dataset: wi
    Shape: (512, 512)
    Dtype: float64
    Min: -3.134147e+01, Max: 3.827918e+01


## Load and Visualize Data


In [3]:
def load_h5_data(filename, dataset_name=None):
    """Load data from an HDF5 file."""
    filepath = os.path.join(data_dir, filename)
    if not os.path.exists(filepath):
        print(f"File {filename} does not exist")
        return None
    
    try:
        with h5py.File(filepath, 'r') as f:
            if dataset_name is None:
                # Try to find the first dataset
                keys = list(f.keys())
                if keys:
                    dataset_name = keys[0]
                else:
                    print(f"No datasets found in {filename}")
                    return None
            
            data = f[dataset_name][:]
            print(f"Loaded {dataset_name} from {filename}")
            print(f"  Shape: {data.shape}")
            print(f"  Dtype: {data.dtype}")
            return data
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        return None

# Load data from each file
den_data = load_h5_data('den.h5')
phi_data = load_h5_data('phi.h5')
w_data = load_h5_data('w.h5')


Loaded den from den.h5
  Shape: (5001, 512, 512)
  Dtype: float64
Loaded phi from phi.h5
  Shape: (5001, 512, 512)
  Dtype: float64
Loaded w from w.h5
  Shape: (5001, 512, 512)
  Dtype: float64


In [ ]:
def plot_2d_field(data, title, cmap='viridis', xlabel='x', ylabel='y'):
    """Plot a 2D field."""
    if data is None:
        print("No data to plot")
        return
    
    plt.figure(figsize=(10, 8))
    
    # Handle different data shapes
    if len(data.shape) == 2:
        # Single 2D field
        im = plt.imshow(data, cmap=cmap, origin='lower', aspect='auto')
        plt.colorbar(im, label=title)
    elif len(data.shape) == 3:
        # Time series of 2D fields - plot first frame
        im = plt.imshow(data[0], cmap=cmap, origin='lower', aspect='auto')
        plt.colorbar(im, label=title)
        plt.title(f"{title} (frame 0)")
    else:
        print(f"Unexpected data shape: {data.shape}")
        return
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Plot the fields
if den_data is not None:
    plot_2d_field(den_data, 'Density (den)', cmap='plasma')

if phi_data is not None:
    plot_2d_field(phi_data, 'Potential (phi)', cmap='RdBu')

if w_data is not None:
    plot_2d_field(w_data, 'Vorticity (w)', cmap='seismic')


## Time Series Analysis


In [ ]:
def analyze_time_series(data, name):
    """Analyze time series data."""
    if data is None:
        return
    
    if len(data.shape) == 3:
        # Calculate statistics over time
        nframes = data.shape[0]
        nx, ny = data.shape[1], data.shape[2]
        
        # Calculate mean, std, min, max over space for each time step
        mean_over_space = np.mean(data, axis=(1, 2))
        std_over_space = np.std(data, axis=(1, 2))
        min_over_space = np.min(data, axis=(1, 2))
        max_over_space = np.max(data, axis=(1, 2))
        
        # Time array (assuming dt=0.005 and nfdump=100 from phw.in)
        dt = 0.005
        nfdump = 100
        time = np.arange(nframes) * dt * nfdump
        
        # Plot statistics
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        axes[0, 0].plot(time, mean_over_space)
        axes[0, 0].set_xlabel('Time')
        axes[0, 0].set_ylabel('Mean')
        axes[0, 0].set_title(f'{name} - Mean over space')
        axes[0, 0].grid(True)
        
        axes[0, 1].plot(time, std_over_space)
        axes[0, 1].set_xlabel('Time')
        axes[0, 1].set_ylabel('Std Dev')
        axes[0, 1].set_title(f'{name} - Std Dev over space')
        axes[0, 1].grid(True)
        
        axes[1, 0].plot(time, min_over_space, label='Min')
        axes[1, 0].plot(time, max_over_space, label='Max')
        axes[1, 0].set_xlabel('Time')
        axes[1, 0].set_ylabel('Value')
        axes[1, 0].set_title(f'{name} - Min/Max over space')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
        
        # Energy (L2 norm)
        energy = np.sum(data**2, axis=(1, 2))
        axes[1, 1].plot(time, energy)
        axes[1, 1].set_xlabel('Time')
        axes[1, 1].set_ylabel('Energy (L2 norm)')
        axes[1, 1].set_title(f'{name} - Energy')
        axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n{name} Statistics:")
        print(f"  Number of frames: {nframes}")
        print(f"  Grid size: {nx} × {ny}")
        print(f"  Time range: {time[0]:.3f} to {time[-1]:.3f}")
        print(f"  Mean value range: [{np.min(mean_over_space):.6e}, {np.max(mean_over_space):.6e}]")
        print(f"  Std Dev range: [{np.min(std_over_space):.6e}, {np.max(std_over_space):.6e}]")

# Analyze time series for each variable
analyze_time_series(den_data, 'Density')
analyze_time_series(phi_data, 'Potential')
analyze_time_series(w_data, 'Vorticity')


## Spatial Analysis


In [ ]:
def plot_multiple_frames(data, name, nframes=4, cmap='viridis'):
    """Plot multiple time frames."""
    if data is None or len(data.shape) != 3:
        print("Need 3D data (time, x, y)")
        return
    
    nframes_available = data.shape[0]
    nframes_to_plot = min(nframes, nframes_available)
    
    # Select evenly spaced frames
    frame_indices = np.linspace(0, nframes_available - 1, nframes_to_plot, dtype=int)
    
    fig, axes = plt.subplots(1, nframes_to_plot, figsize=(5*nframes_to_plot, 5))
    if nframes_to_plot == 1:
        axes = [axes]
    
    for i, frame_idx in enumerate(frame_indices):
        im = axes[i].imshow(data[frame_idx], cmap=cmap, origin='lower', aspect='auto')
        axes[i].set_title(f'{name}\nFrame {frame_idx}')
        axes[i].set_xlabel('x')
        if i == 0:
            axes[i].set_ylabel('y')
        plt.colorbar(im, ax=axes[i])
    
    plt.tight_layout()
    plt.show()

# Plot multiple frames
if den_data is not None and len(den_data.shape) == 3:
    plot_multiple_frames(den_data, 'Density', nframes=4, cmap='plasma')

if phi_data is not None and len(phi_data.shape) == 3:
    plot_multiple_frames(phi_data, 'Potential', nframes=4, cmap='RdBu')

if w_data is not None and len(w_data.shape) == 3:
    plot_multiple_frames(w_data, 'Vorticity', nframes=4, cmap='seismic')


## Fourier Analysis


In [ ]:
def plot_power_spectrum(data, name, frame_idx=0):
    """Plot power spectrum of a 2D field."""
    if data is None:
        return
    
    # Get 2D field
    if len(data.shape) == 3:
        field = data[frame_idx]
    elif len(data.shape) == 2:
        field = data
    else:
        print(f"Unexpected data shape: {data.shape}")
        return
    
    # Compute 2D FFT
    fft_field = np.fft.fft2(field)
    power_spectrum = np.abs(fft_field)**2
    
    # Shift zero frequency to center
    power_spectrum_shifted = np.fft.fftshift(power_spectrum)
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Original field
    im1 = axes[0].imshow(field, cmap='viridis', origin='lower', aspect='auto')
    axes[0].set_title(f'{name} - Original Field')
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    plt.colorbar(im1, ax=axes[0])
    
    # Power spectrum (log scale)
    im2 = axes[1].imshow(np.log10(power_spectrum_shifted + 1e-10), 
                         cmap='hot', origin='lower', aspect='auto')
    axes[1].set_title(f'{name} - Power Spectrum (log scale)')
    axes[1].set_xlabel('kx')
    axes[1].set_ylabel('ky')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()

# Plot power spectra
if den_data is not None:
    plot_power_spectrum(den_data, 'Density')

if phi_data is not None:
    plot_power_spectrum(phi_data, 'Potential')

if w_data is not None:
    plot_power_spectrum(w_data, 'Vorticity')


## Correlation Analysis


In [ ]:
def plot_correlation(data1, data2, name1, name2, frame_idx=0):
    """Plot correlation between two fields."""
    if data1 is None or data2 is None:
        print("Both datasets needed for correlation")
        return
    
    # Get 2D fields
    if len(data1.shape) == 3:
        field1 = data1[frame_idx]
    else:
        field1 = data1
    
    if len(data2.shape) == 3:
        field2 = data2[frame_idx]
    else:
        field2 = data2
    
    # Flatten and compute correlation
    flat1 = field1.flatten()
    flat2 = field2.flatten()
    
    correlation = np.corrcoef(flat1, flat2)[0, 1]
    
    # Plot scatter
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Scatter plot
    axes[0].scatter(flat1, flat2, alpha=0.5, s=1)
    axes[0].set_xlabel(name1)
    axes[0].set_ylabel(name2)
    axes[0].set_title(f'Correlation: {correlation:.4f}')
    axes[0].grid(True)
    
    # Overlay fields
    axes[1].imshow(field1, cmap='RdBu', origin='lower', aspect='auto', alpha=0.5)
    axes[1].imshow(field2, cmap='viridis', origin='lower', aspect='auto', alpha=0.5)
    axes[1].set_title(f'{name1} (red/blue) and {name2} (green) overlay')
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')
    
    plt.tight_layout()
    plt.show()
    
    return correlation

# Plot correlations
if den_data is not None and phi_data is not None:
    corr_den_phi = plot_correlation(den_data, phi_data, 'Density', 'Potential')
    print(f"Density-Potential correlation: {corr_den_phi:.4f}")

if phi_data is not None and w_data is not None:
    corr_phi_w = plot_correlation(phi_data, w_data, 'Potential', 'Vorticity')
    print(f"Potential-Vorticity correlation: {corr_phi_w:.4f}")


## Custom Analysis

Add your own analysis code below:


In [ ]:
# Example: Access specific frames or time slices
# frame_number = 100
# if den_data is not None and len(den_data.shape) == 3:
#     specific_frame = den_data[frame_number]
#     print(f"Frame {frame_number} shape: {specific_frame.shape}")

# Example: Calculate zonal averages
# if den_data is not None:
#     if len(den_data.shape) == 3:
#         zonal_avg = np.mean(den_data, axis=2)  # Average over y
#     else:
#         zonal_avg = np.mean(den_data, axis=1)
#     print(f"Zonal average shape: {zonal_avg.shape}")

# Add your custom analysis here
